# Resample hourly to daily

This notebook is used for resampling the restacked 20km outputs to daily files. 

We will submit a slurm job for each hourly file that will then be resampled to a daily resolution.

In [5]:
import xarray as xr
from config import *
import slurm

In [2]:
restack_scratch_dir

PosixPath('/center1/DYNDOWN/kmredilla/wrf_data/restacked')

In [ ]:
out_varnames = ["t2", "t2min", "t2max", "q2", "pcpc", "pcpt"]

sbatch_fps = []
for varname in out_varnames:
    for year in years:
        
        wrf_varname = luts.resample_varnames["wrf_varname"][varname]
        # write to .slurm script
        sbatch_fp = slurm_dir.joinpath(f"resample_{group}_{year}_{varname}.slurm")
        # filepath for slurm stdout
        sbatch_out_fp = slurm_dir.joinpath(f"resample_{group}_{year}_{varname}_%j.out")
        sbatch_head = slurm.make_sbatch_head(
            slurm_email, partition, conda_init_script
        )

        args = {
            "sbatch_fp": sbatch_fp,
            "sbatch_out_fp": sbatch_out_fp,
            "resample_script": resample_script,
            "luts_fp": luts_fp,
            "restacked_dir": restack_scratch_dir,
            "group": group,
            "fn_str": luts.groups[group]["fn_str"],
            "varname": varname,
            "ncpus": ncpus,
            "sbatch_head": sbatch_head,
        }

        slurm.write_sbatch_resample(**args)
        sbatch_fps.append(sbatch_fp)

In [9]:
with xr.open_dataset("/center1/DYNDOWN/kmredilla/wrf_data/restacked/t2/t2_hourly_wrf_NCAR-CCSM4_historical_1970.nc") as ds:
    test = ds["t2"].resample(time="1D").mean()

210

In [7]:
ds["t2"

<xarray.Dataset>
Dimensions:              (time: 8736, yc: 262, xc: 262)
Coordinates:
  * time                 (time) datetime64[ns] 1970-01-02 ... 1970-12-31T23:0...
  * yc                   (yc) float64 -1.824e+05 -2.024e+05 ... -5.402e+06
  * xc                   (xc) float64 -2.61e+06 -2.59e+06 ... 2.59e+06 2.61e+06
    lon                  (yc, xc) float32 ...
    lat                  (yc, xc) float32 ...
Data variables:
    t2                   (time, yc, xc) float32 ...
    polar_stereographic  |S1 ...
Attributes:
    Conventions:      None
    proj_parameters:  +proj=stere +units=m +a=6370000.0 +b=6370000.0 +lat_0=9...
    restacked_by:     Scenarios Network for Alaska + Arctic Planning -- 2022
    version:          1.1
    history:          creation system: Linux m79 2.6.32-431.29.2.el6.658g0000...